In [ ]:
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from __future__ import division
from __future__ import print_function

import os
import sys
import pandas as pd
import numpy as np
import random
from collections import Counter
import tensorflow as tf
# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
from pyod.models.copod import COPOD
from pyod.models.ecod import ECOD
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy, Precision, AUC, Recall

In [ ]:
X = pd.read_csv('/content/drive/MyDrive/one_hot_data.csv')

In [ ]:
X.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
X = X.iloc[:, 1:]


In [ ]:
X.head(5)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/X_train1_oh.csv')
X_test = pd.read_csv('/content/drive/MyDrive/X_test_oh.csv')

In [ ]:
X_train.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
X_train.shape

(140250, 385)

In [ ]:
X_train=X_train.iloc[:, 1:]
X_test=X_test.iloc[:, 1:]

In [ ]:
X_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
X_train = X_train.dropna().reset_index(drop=True)

In [ ]:
X_train.head

<bound method NDFrame.head of           0    1    2    3    4    5    6    7    8    9  ...  374  375  376  \
0       1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   
1       1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   
2       0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  1.0  1.0   
3       0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   
4       1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
140245  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  1.0  1.0  1.0   
140246  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  1.0  1.0  1.0   
140247  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   
140248  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   
140249  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   

        3

In [ ]:
X_test.head(5)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
X_test.isnull().values.any()

False

In [ ]:
#Scale the data using standard scaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X.shape

(350626, 384)

In [ ]:
from sklearn.decomposition import PCA
# Loop Function to identify number of principal components that explain at least 85% of the variance
for comp in range(3, X.shape[1]):
    pca = PCA(n_components= comp, random_state=42)
    pca.fit(X)
    comp_check = pca.explained_variance_ratio_
    final_comp = comp
    if comp_check.sum() > 0.85:
        break
        
Final_PCA = PCA(n_components= final_comp,random_state=42)
Final_PCA.fit(X)
cluster_df=Final_PCA.transform(X)
num_comps = comp_check.shape[0]
print("Using {} components, we can explain {}% of the variability in the original data.".format(final_comp,comp_check.sum()))

Using 24 components, we can explain 0.8536882077097732% of the variability in the original data.


In [ ]:
random.shuffle(cluster_df)

In [ ]:
#Splitting the data to train and test

split_len = 0.4 * len(X)
split_len
X_train = cluster_df[:int(split_len)]
X_test = cluster_df[int(split_len):]

In [ ]:
X1 = pd.DataFrame(X_test)

In [ ]:
X1.head(5)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.008865,-0.042877,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218
1,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
2,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883
3,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883
4,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883


In [ ]:
X1.shape

(210376, 24)

In [ ]:
if __name__ == "__main__":
  
    # train COPOD detector
    clf_name = 'COPOD'
    clf = COPOD()

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(X_train)

    # get the prediction labels and outlier scores of the training data
    y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    # get the prediction on the test data
    y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
    y_test_scores = clf.decision_function(X_test)  # outlier scores


In [ ]:
Counter(y_train_pred)

Counter({0: 126910, 1: 13340})

In [ ]:
Counter(y_test_pred)

Counter({0: 192552, 1: 17824})

In [ ]:
number_of_testoutliers = len(y_test_pred[y_test_pred == 1])

In [ ]:
number_of_testoutliers

17824

In [ ]:
type(y_test_pred)

numpy.ndarray

In [ ]:
arr = np.array([y_test_pred])
list1 = arr.tolist()
print(list1)
print(type(list1))

[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,

In [ ]:
cols= ['label']
X2 = pd.DataFrame(y_test_pred, columns=cols)

In [ ]:
label_X = X2['label']
X_test2 = X1.join(label_X)

In [ ]:
X_test2.shape

(210376, 25)

In [ ]:
p2 = X_test2[X_test2.label == 0]

In [ ]:
p2.shape

(192552, 25)

In [ ]:
split_len = 0.7 * len(p2)
split_len

134786.4

In [ ]:
auto2 = p2[:int(split_len)]
autotest = p2[int(split_len):]

In [ ]:
auto2.shape

(134786, 25)

In [ ]:
trainp = auto2.drop(['label'], axis=True)

In [ ]:
trainp.shape

(134786, 24)

In [ ]:
autotest = autotest.drop(['label'], axis=True)

In [ ]:
autotest.shape

(57766, 25)

In [ ]:
p3 = X_test2[X_test2.label == 1]

In [ ]:
p3 = p3.drop(['label'], axis=1)

In [ ]:
p3.shape

(17824, 24)

In [ ]:
frames = [autotest, p3]

In [ ]:
autoentest = pd.concat(frames)

In [ ]:
autoentest.shape

(75590, 25)

In [ ]:
autoentest = autoentest.drop(['label'], axis = 1)

In [ ]:
autoentest.head(5)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
147205,-3.237478,0.572792,-0.140058,-0.019660,0.000501,0.347647,-1.298052,-0.159948,-0.199584,-2.404718,...,0.008423,0.030862,-0.004234,0.001502,-0.007928,-0.000122,-0.005261,0.000057,-0.000095,0.003253
147206,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
147207,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
147208,12.470189,-3.865843,1.147876,0.209206,-0.027616,-3.626393,-1.039736,-9.173712,0.109152,-0.023301,...,-0.000744,-0.008945,0.142693,0.013884,-0.835312,-5.068386,0.000642,-2.002038,-8.150398,-0.161860
147209,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.008865,-0.042877,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218


In [ ]:
testp = autoentest.reset_index(drop=True)

In [ ]:
testp.shape

(75590, 24)

In [ ]:
#Test set for autoencoder
testp.head(5)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-3.237478,0.572792,-0.140058,-0.019660,0.000501,0.347647,-1.298052,-0.159948,-0.199584,-2.404718,...,0.008423,0.030862,-0.004234,0.001502,-0.007928,-0.000122,-0.005261,0.000057,-0.000095,0.003253
1,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
2,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
3,12.470189,-3.865843,1.147876,0.209206,-0.027616,-3.626393,-1.039736,-9.173712,0.109152,-0.023301,...,-0.000744,-0.008945,0.142693,0.013884,-0.835312,-5.068386,0.000642,-2.002038,-8.150398,-0.161860
4,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.008865,-0.042877,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218


In [ ]:
#Training set for autoencoder
trainp.shape

(134786, 24)

In [ ]:
trainp.head(5)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.008865,-0.042877,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218
1,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
2,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883
3,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883
4,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883


In [ ]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    print(output_units)
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),
          #Reducing the dimensionality of features from 384 to 24
          Dense(24, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
auto_encoder2 = AutoEncoders(len(trainp.iloc[0]))

auto_encoder2.compile(
    loss='mean_squared_error',
    metrics=['mae', 'acc', tf.keras.metrics.Precision(), tf.keras.metrics.AUC(), tf.keras.metrics.Recall()],
    optimizer='adam'
)

#Must pass the training set as both input and output 
#as the encoder will learn to compress the dataset 
history = auto_encoder2.fit(
    trainp, 
    trainp, 
    epochs=60, 
    batch_size=1000,
    #validation_data=(x_test_scaled, x_test_scaled)

)


24
Epoch 1/60
135/135 [==============================] - 4s 16ms/step - loss: 3.6661 - mae: 0.7800 - acc: 0.8759 - precision: 1.0000 - auc: 0.0000e+00 - recall: 0.1196
Epoch 2/60
135/135 [==============================] - 2s 15ms/step - loss: 3.5647 - mae: 0.7107 - acc: 0.9998 - precision: 1.0000 - auc: 0.0000e+00 - recall: 0.1052
Epoch 3/60
135/135 [==============================] - 2s 14ms/step - loss: 3.5645 - mae: 0.7100 - acc: 1.0000 - precision: 1.0000 - auc: 0.0000e+00 - recall: 0.1052
Epoch 4/60
135/135 [==============================] - 2s 16ms/step - loss: 3.5645 - mae: 0.7099 - acc: 1.0000 - precision: 1.0000 - auc: 0.0000e+00 - recall: 0.1052
Epoch 5/60
135/135 [==============================] - 2s 13ms/step - loss: 3.5645 - mae: 0.7098 - acc: 1.0000 - precision: 1.0000 - auc: 0.0000e+00 - recall: 0.1052
Epoch 6/60
135/135 [==============================] - 2s 15ms/step - loss: 3.5645 - mae: 0.7098 - acc: 1.0000 - precision: 1.0000 - auc: 0.0000e+00 - recall: 0.1052
Epoch 7

In [ ]:
auto_encoder2.summary()

Model: "auto_encoders"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 24)                25112     
                                                                 
 sequential_1 (Sequential)   (None, 24)                12568     
                                                                 
Total params: 37,680
Trainable params: 37,680
Non-trainable params: 0
_________________________________________________________________


In [ ]:
encoder_layer = auto_encoder2.get_layer('sequential')
o1 = pd.DataFrame(encoder_layer.predict(trainp))
o1 = trainp.add_prefix('new_')

In [ ]:
o1.shape

(134786, 24)

In [ ]:
o1.head(5)

,new_0,new_1,new_2,new_3,new_4,new_5,new_6,new_7,new_8,new_9,...,new_14,new_15,new_16,new_17,new_18,new_19,new_20,new_21,new_22,new_23
0,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.008865,-0.042877,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218
1,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
2,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883
3,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883
4,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.006399,-0.891665,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883


In [ ]:
#Compute average and standard deviation of feature values of each sample
o1['o1_mean'] = o1.mean(axis = 1) 
o1['std'] = o1.std(axis = 1)
o1_std = o1['std'] = o1.std(axis = 1)

In [ ]:
o1.head(5)

,new_0,new_1,new_2,new_3,new_4,new_5,new_6,new_7,new_8,new_9,...,new_16,new_17,new_18,new_19,new_20,new_21,new_22,new_23,o1_mean,std
0,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218,0.248533,1.741842
1,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680,-0.184776,1.467803
2,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883,-0.281666,1.458842
3,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883,-0.281666,1.458842
4,-3.207634,0.576783,-0.142829,-0.020752,0.000780,0.404387,-2.926345,-0.083574,-0.232502,-3.391124,...,-0.018555,-0.002192,-0.019194,-0.000269,-0.004393,0.000108,-0.000174,0.005883,-0.281666,1.458842


In [ ]:
#Calculating the threshold is done by multiplying std with a constant c.
#The c is chosen based on the predicted percentage of negative logs in the prediction of IF/COPOD/ECOD
threshold_value = o1_std * 0.084

In [ ]:
print(threshold_value[5])

0.1225427610265783


In [ ]:
encoder_layer = auto_encoder2.get_layer('sequential')
o2 = pd.DataFrame(encoder_layer.predict(autoentest))
o2 = testp.add_prefix('new_')

In [ ]:
o2.shape

(75590, 24)

In [ ]:
o2.head(5)

,new_0,new_1,new_2,new_3,new_4,new_5,new_6,new_7,new_8,new_9,...,new_14,new_15,new_16,new_17,new_18,new_19,new_20,new_21,new_22,new_23
0,-3.237478,0.572792,-0.140058,-0.019660,0.000501,0.347647,-1.298052,-0.159948,-0.199584,-2.404718,...,0.008423,0.030862,-0.004234,0.001502,-0.007928,-0.000122,-0.005261,0.000057,-0.000095,0.003253
1,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
2,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.067078,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680
3,12.470189,-3.865843,1.147876,0.209206,-0.027616,-3.626393,-1.039736,-9.173712,0.109152,-0.023301,...,-0.000744,-0.008945,0.142693,0.013884,-0.835312,-5.068386,0.000642,-2.002038,-8.150398,-0.161860
4,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.008865,-0.042877,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218


In [ ]:
o2['o2_mean'] = o2.mean(axis = 1) 

In [ ]:
o2.head(5)

,new_0,new_1,new_2,new_3,new_4,new_5,new_6,new_7,new_8,new_9,...,new_15,new_16,new_17,new_18,new_19,new_20,new_21,new_22,new_23,o2_mean
0,-3.237478,0.572792,-0.140058,-0.019660,0.000501,0.347647,-1.298052,-0.159948,-0.199584,-2.404718,...,0.030862,-0.004234,0.001502,-0.007928,-0.000122,-0.005261,0.000057,-0.000095,0.003253,-0.312914
1,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680,-0.184776
2,-3.156321,0.563280,-0.139377,-0.020823,0.002107,0.395610,-3.153685,-0.025669,0.199920,4.732233,...,-0.131518,-0.016090,-0.012534,-0.017931,-0.000254,-0.013866,0.000101,-0.000159,0.005680,-0.184776
3,12.470189,-3.865843,1.147876,0.209206,-0.027616,-3.626393,-1.039736,-9.173712,0.109152,-0.023301,...,-0.008945,0.142693,0.013884,-0.835312,-5.068386,0.000642,-2.002038,-8.150398,-0.161860,-0.830096
4,-2.245255,0.256481,-0.045405,-0.000843,-0.001860,-0.217780,8.309989,-0.353633,-0.002147,0.440913,...,-0.042877,-0.024338,-0.006294,-0.026331,-0.000371,-0.000956,0.000150,-0.000243,0.008218,0.248533


In [ ]:
#If the mean of the o2 dataset is less than the threshold then that row is anamoly
o2.loc[o2['o2_mean'] < 0.1225427610265783, 'label'] = 'Anamoly'  
o2.loc[o2['o2_mean'] > 0.1225427610265783, 'label'] = 'not_Anamoly'  

In [ ]:
o2.head

<bound method NDFrame.head of            new_0      new_1      new_2      new_3     new_4      new_5  \
0      -3.237478   0.572792  -0.140058  -0.019660  0.000501   0.347647   
1      -3.156321   0.563280  -0.139377  -0.020823  0.002107   0.395610   
2      -3.156321   0.563280  -0.139377  -0.020823  0.002107   0.395610   
3      12.470189  -3.865843   1.147876   0.209206 -0.027616  -3.626393   
4      -2.245255   0.256481  -0.045405  -0.000843 -0.001860  -0.217780   
...          ...        ...        ...        ...       ...        ...   
75585  33.578217  85.654101 -54.102183 -66.760821  0.271733 -15.822842   
75586   9.370783  -2.949425   0.864195   0.200933 -0.013998  -2.982073   
75587  10.317100  -3.274573   0.963037   0.233963 -0.000659  -3.400803   
75588  12.445956  -3.859287   1.146452   0.209866 -0.027653  -3.665836   
75589  10.478233  -3.335914   0.990589   0.210073 -0.019686  -3.350962   

          new_6     new_7     new_8     new_9  ...     new_16    new_17  \
0     

In [ ]:
result = o2[o2.label == 'not_Anamoly']

In [ ]:
result.shape

(32792, 26)